## Data Processing Part3

data sources 

https://en.wikipedia.org/wiki/List_of_American_films_of_2018

https://en.wikipedia.org/wiki/List_of_American_films_of_2019

https://en.wikipedia.org/wiki/List_of_American_films_of_2020

In [31]:
!pip install tmdbv3api

In [56]:
!pip install pyyaml

In [7]:
import pandas as pd
import numpy as np
import ast

import matplotlib.pyplot as ply
import seaborn as sns

from tmdbv3api import TMDb
from tmdbv3api import Movie
import json
import requests

import yaml

In [290]:

# Function to get genres
def get_genre(x, tmdb, tmdb_movie): #pass in the title of the movies
    try:
        genres = []
        result = tmdb_movie.search(x) #the title will be searched in the tmdb_movie
        movie_id = result[0].id #we will match the "id" with the "title"
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key)) #we will get the result from the IMDb data
        data_json = response.json() # we will then convert it to a json file
        if data_json['genres']: #in the json file we will only need to extract the "genre"
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name']) #we will then add the "genre" to the empty genre list we created above
            return genre_str.join(genres)
        else:
            np.NaN # we will return the results but if we don't find anything we will consider it as a missing value
    except Exception as e:
        return np.NaN


# Function to get directors name
def get_director(x):
    if " (director)" in x: # we are getting "director"
        return x.split(" (director)")[0]
    elif " (directors)" in x: # we are getting "directors" with "s"
        return x.split(" (directors)")[0]
    elif " (director[" in x:
        return x.split(" (director[")[0]
    elif " (director/" in x:
        return x.split(" (director/")[0] # we are getting "director/screenplay"
        
# Functions to get actor names
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])
    
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])


# Final data processing function
def ProcessData(df1, df2, df3, df4, config_file = "config.yml"):
    
    # Append all 4 dataframes together
    df = pd.concat([df1, df2, df3, df4], axis = 0, ignore_index=True)

    df = df.dropna(subset=['Title'])

    if df.Title.str.contains('closure of theaters caused by the COVID-19 pandemic').any():
        df = df[~(df.Title.str.contains('closure of theaters caused by the COVID-19 pandemic'))]

    if df.Title.str.contains('Canadian theatrical release').any():
        df = df[~(df.Title.str.contains('Canadian theatrical release'))]

    if df.Title.str.contains('Dating and New York').any():
        df = df[(df.Title!= 'Dating and New York')]

    if df.Title.str.contains('Apollo 10').any():
        df = df[~(df.Title.str.contains('Apollo 10'))]


    # Get TMDB API key
    # get your API Key here: https://www.themoviedb.org/settings/api by creating an account and requesting API access
    
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    
    api_key = config["api_key"]

    tmdb = TMDb()
    tmdb.api_key = api_key 
    tmdb_movie = Movie()

    # Get genres based on Title using TMDB ping
    df['genres'] = df['Title'].map(lambda x: get_genre(str(x), tmdb, tmdb_movie))

    df_sub = df[['Title', 'Cast and crew', 'genres']].copy()

    # Get director's name from "Cast and crew" column
    df_sub['director_name'] = df_sub['Cast and crew'].map(lambda x: get_director(x))
    df_sub['director_name'] = df_sub['director_name'].str.replace(" (director/screenplay);",",")

    # Get actor 1 name
    df_sub['actor_1_name'] = df_sub['Cast and crew'].map(lambda x: get_actor1(x))
    df_sub['actor_1_name'] = df_sub['actor_1_name'].str.replace("Lauren Wolkstein (director); ", "")

    # Get actor 2 name
    df_sub['actor_2_name'] = df_sub['Cast and crew'].map(lambda x: get_actor2(x))

    # Get actor 3 name
    df_sub['actor_3_name'] = df_sub['Cast and crew'].map(lambda x: get_actor3(x))

    # rename title col
    df_sub = df_sub.rename(columns={'Title':'movie_title'})

    # Final Cleaning and preparation
    new_df_sub = df_sub.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
    new_df_sub['actor_1_name'] = new_df_sub['actor_1_name'].replace(np.nan, 'unknown')
    new_df_sub['actor_2_name'] = new_df_sub['actor_2_name'].replace(np.nan, 'unknown')
    new_df_sub['actor_3_name'] = new_df_sub['actor_3_name'].replace(np.nan, 'unknown')
    new_df_sub['movie_title'] = new_df_sub['movie_title'].str.lower()
    new_df_sub['comb'] = new_df_sub['actor_1_name'] + ' ' + new_df_sub['actor_2_name'] + ' '+ new_df_sub['actor_3_name'] + ' '+ new_df_sub['director_name'] +' ' + new_df_sub['genres']

    return new_df_sub


### 2018 data

In [9]:
# Read data from html

link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [11]:
# Peek into data downloaded from html

display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,J A N U A R Y,5,The Strange Ones,Vertical Entertainment,Christopher Radcliff (director/screenplay); La...,[3]
2,J A N U A R Y,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[4]
3,J A N U A R Y,12,Proud Mary,Screen Gems,"Babak Najafi (director); John S. Newman, Chris...",[5]
4,J A N U A R Y,12,Acts of Violence,Lionsgate Premiere,Brett Donowho (director); Nicolas Aaron Mezzan...,[6]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,A P R I L,6,A Quiet Place,Paramount Pictures / Platinum Dunes / Sunday N...,John Krasinski (director/screenplay); Bryan Wo...,[60]
1,A P R I L,6,Blockers,Universal Pictures / Good Universe / Point Gre...,"Kay Cannon (director); Brian Kehoe, Jim Kehoe ...",[61]
2,A P R I L,6,You Were Never Really Here,Amazon Studios,Lynne Ramsay (director/screenplay); Joaquin Ph...,[62]
3,A P R I L,6,Chappaquiddick,Entertainment Studios,"John Curran (director); Taylor Allen, Andrew L...",[63]
4,A P R I L,6,The Miracle Season,LD Entertainment,"Sean McNamara (director); David Aaron Cohen, E...",[64]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J U L Y,3,Who's Watching Oliver,Motionpictures / Gravitas Ventures,Richie Moore (director/screenplay); Raimund Hu...,[114]
1,J U L Y,4,The First Purge,Universal Pictures / Blumhouse Productions / P...,Gerard McMurray (director); James DeMonaco (sc...,[115]
2,J U L Y,6,Ant-Man and the Wasp,Marvel Studios,"Peyton Reed (director); Chris McKenna, Erik So...",[116]
3,J U L Y,6,Sorry to Bother You,Annapurna Pictures,Boots Riley (director/screenplay); LaKeith Sta...,[117]
4,J U L Y,6,Whitney,Roadside Attractions / Miramax,Kevin Macdonald (director/screenplay); Whitney...,[118]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,O C T O B E R,5,Venom,Columbia Pictures / Marvel Entertainment,"Ruben Fleischer (director); Jeff Pinkner, Scot...",[182]
1,O C T O B E R,5,A Star Is Born,Warner Bros. Pictures / Peters Entertainment,Bradley Cooper (director/screenplay); Eric Rot...,[183]
2,O C T O B E R,5,The Great Buster: A Celebration,Cohen Media Group,Peter Bogdanovich (director/screenplay),[184]
3,O C T O B E R,12,First Man,Universal Pictures / DreamWorks Pictures / Amb...,Damien Chazelle (director); Josh Singer (scree...,[185]
4,O C T O B E R,12,Bad Times at the El Royale,20th Century Fox,Drew Goddard (director/screenplay); Jeff Bridg...,[186]


In [35]:
new_df18 = ProcessData(df1, df2, df3, df4, config_file = "config.yml")
new_df18.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,"Christopher Radcliff, Lauren Wolkstein",Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Drama Mystery,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
3,Babak Najafi,Taraji P. Henson,Jahi Di'Allo Winston,Billy Brown,Thriller Action Crime,proud mary,Taraji P. Henson Jahi Di'Allo Winston Billy Br...
4,Brett Donowho,Bruce Willis,Cole Hauser,Shawn Ashmore,Action Crime Thriller,acts of violence,Bruce Willis Cole Hauser Shawn Ashmore Brett D...


### 2019 data

In [41]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [43]:
# Peek into data downloaded from html

display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,J A N U A R Y,4,Rust Creek,IFC Films / Lunacy Productions,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,J A N U A R Y,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,J A N U A R Y,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,J A N U A R Y,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,A P R I L,5,Shazam!,Warner Bros. Pictures / New Line Cinema / DC F...,David F. Sandberg (director); Henry Gayden (sc...,[51]
1,A P R I L,5,Pet Sematary,"Paramount Pictures / Room 101, Inc.","Kevin Kolsch, Dennis Widmyer (directors); Jeff...",[52]
2,A P R I L,5,The Best of Enemies,STX Entertainment,Robin Bissell (director/screenplay); Taraji P....,[53]
3,A P R I L,5,High Life,A24 / Arte / ZDF / Canal+ / Ciné+,Claire Denis (director/screenplay); Jean-Pol F...,[54]
4,A P R I L,5,Unicorn Store,Netflix,Brie Larson (director); Samantha McIntyre (scr...,NaN


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J U L Y,5,Spider-Man: Far From Home,Columbia Pictures / Marvel Studios,"Jon Watts (director); Chris McKenna, Erik Somm...",[115]
1,J U L Y,5,Escape Plan: The Extractors,Summit Entertainment,"John Herzfeld (director); John Herzfeld, Miles...",[116]
2,J U L Y,12,Midsommar,A24,Ari Aster (director/screenplay); Florence Pugh...,[117]
3,J U L Y,12,Stuber,20th Century Fox,Michael Dowse (director); Tripper Clancy (scre...,[118]
4,J U L Y,12,Crawl,Paramount Pictures / Ghost House Pictures,"Alexandre Aja (director); Shawn Rasmussen, Mic...",[119]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,O C T O B E R,4,Joker,Warner Bros. Pictures / Village Roadshow Pictu...,Todd Phillips (director/screenplay); Scott Sil...,[171]
1,O C T O B E R,4,Lucy in the Sky,Fox Searchlight Pictures,Noah Hawley (director/screenplay); Brian C. Br...,[172]
2,O C T O B E R,4,Dolemite Is My Name,Netflix,Craig Brewer (director); Scott Alexander and L...,[167]
3,O C T O B E R,4,In the Tall Grass,Netflix,Vincenzo Natali (director/screenplay); Patrick...,NaN
4,O C T O B E R,4,Low Tide,A24 / DirecTV Cinema,Kevin McMullin (director/screenplay); Keean Jo...,[173]


In [45]:
new_df19 = ProcessData(df1, df2, df3, df4, config_file = "config.yml")
new_df19.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Horror Thriller Mystery,escape room,Taylor Russell Logan Miller Deborah Ann Woll A...
1,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Thriller Drama Action Crime,rust creek,Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Thriller,american hangman,Donald Sutherland Vincent Kartheiser Oliver De...
3,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Drama Adventure Family,a dog's way home,Bryce Dallas Howard Edward James Olmos Alexand...
4,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Comedy Drama,the upside,Bryan Cranston Kevin Hart Nicole Kidman Neil B...


### 2020 data

In [127]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [129]:
# Peek into data downloaded from html

display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,J A N U A R Y,10,Inherit the Viper,Lionsgate / Barry Films / Tycor International ...,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,A P R I L,3,Coffee & Kareem,Netflix,Michael Dowse (director); Shane McCarthy (scre...,[57]
1,A P R I L,10,Trolls World Tour,Universal Pictures / DreamWorks Animation,"Walt Dohrn (director); Jonathan Aibel, Glenn B...",[58]
2,A P R I L,10,Love Wedding Repeat,Netflix,"Dean Craig (director/screenplay); Olivia Munn,...",[59]
3,A P R I L,10,The Main Event,Netflix / WWE Studios,Jay Karas (director); Larry Postel (screenplay...,[60]
4,A P R I L,10,We Summon the Darkness,Saban Films,Marc Meyers (director); Alan Trezza (screenpla...,[61]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J U L Y,3,Hamilton,Disney+ / Walt Disney Pictures,"Thomas Kail (director); Lin-Manuel Miranda, Ph...",[103]
1,J U L Y,3,Relic,IFC Midnight / Nine Stories Productions / AGBO,Natalie Erika James (director/screenplay); Chr...,[104]
2,J U L Y,3,Desperados,Netflix,LP (director); Ellen Rapoport (screenplay); An...,[105]
3,J U L Y,10,The Old Guard,Netflix / Skydance Media,Gina Prince-Bythewood (director); Greg Rucka (...,[106]
4,J U L Y,10,Money Plane,Quiver Distribution,Andrew Lawrence (director/screenplay); Tim Sch...,[107]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,O C T O B E R,2,On the Rocks,Apple TV+ / A24 / American Zoetrope,Sofia Coppola (director/screenplay); Rashida J...,[169]
1,O C T O B E R,2,Spontaneous,Paramount Pictures / Awesomeness Films / Juras...,Brian Duffield (director/screenplay); Katherin...,[170]
2,O C T O B E R,2,Vampires vs. the Bronx,Netflix / Universal Pictures,Osmany Rodriguez (director/screenplay); Blaise...,[171]
3,O C T O B E R,2,Possessor,Neon / Well Go USA Entertainment / Elevation P...,Brandon Cronenberg (director/screenplay); Andr...,[172]
4,O C T O B E R,2,Death of Me,Saban Films,"Darren Lynn Bousman (director); Arli Margolis,...",[173]


In [114]:
new_df20 = ProcessData(df1, df2, df3, df4, config_file = "config.yml")
new_df20.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...


### 2021 data

In [187]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [188]:
# Peek into data downloaded from html

display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,1,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2]
1,J A N U A R Y,5,Hacksaw,Leone Films / Midnight Releasing,"Anthony Leone (director/screenplay); Amy Cay, ...",[3]
2,J A N U A R Y,12,Dr. Bird's Advice for Sad Poets,Relativity Media / Ketchup Entertainment,Yaniv Raz (director/screenplay); Lucas Jade Zu...,[4]
3,J A N U A R Y,13,The White Tiger,Netflix / ARRAY / Purple Pebble Pictures,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN
4,J A N U A R Y,14,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[5]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,A P R I L,2,Concrete Cowboy,Netflix,Ricky Staub (director/screenplay); Dan Walser ...,[60]
1,A P R I L,2,The Unholy,Screen Gems / Ghost House Pictures,Evan Spiliotopoulos (director/screenplay); Jef...,[61]
2,A P R I L,2,Every Breath You Take,Vertical Entertainment,Vaughn Stein (director); David Murray (screenp...,[62]
3,A P R I L,2,Shiva Baby,Utopia,Emma Seligman (director/screenplay); Rachel Se...,NaN
4,A P R I L,2,The Girl Who Believes in Miracles,Atlas Distribution Company,Richard Correll (director/screenplay); G.M. Me...,NaN


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J U L Y,1,No Sudden Move,HBO Max / Warner Bros. Pictures,Steven Soderbergh (director); Ed Solomon (scre...,[113]
1,J U L Y,2,The Tomorrow War,Amazon Studios / Paramount Pictures / Skydance...,Chris McKay (director); Zach Dean (screenplay)...,[114]
2,J U L Y,2,The Boss Baby: Family Business,Universal Pictures / DreamWorks Animation / Pe...,Tom McGrath (director); Michael McCullers (scr...,[115]
3,J U L Y,2,The Forever Purge,Universal Pictures / Platinum Dunes / Blumhous...,Everardo Gout (director); James DeMonaco (scre...,[116]
4,J U L Y,2,Fear Street Part One: 1994,Netflix / Chernin Entertainment,Leigh Janiak (director/screenplay); Phil Grazi...,NaN


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,O C T O B E R,1.0,Venom: Let There Be Carnage,Columbia Pictures / Marvel Entertainment / Ten...,"Andy Serkis (director), Kelly Marcel (screenpl...",[204]
1,O C T O B E R,1.0,The Many Saints of Newark,Warner Bros. Pictures / New Line Cinema / HBO ...,"Alan Taylor (director); David Chase, Lawrence ...",[191]
2,O C T O B E R,1.0,The Addams Family 2,Metro-Goldwyn-Mayer / Bron Creative,"Conrad Vernon, Greg Tiernan (directors); Dan H...",[205]
3,O C T O B E R,1.0,Bingo Hell,Amazon Studios / Blumhouse Television,Gigi Saul Guerrero (director/screenplay); Shan...,[206]
4,O C T O B E R,1.0,Black as Night,Amazon Studios / Blumhouse Television,Maritte Lee Go (director); Sherman Payne (scre...,[206]


In [207]:
new_df21 = ProcessData(df1, df2, df3, df4, config_file = "config.yml")
new_df21.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,Adventure Family TV Movie Western,hacksaw,Amy Cay Brian Patrick Butler Michael C. Burges...
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,Comedy Drama,dr. bird's advice for sad poets,Lucas Jade Zumann Taylor Russell Chase Stokes ...
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Romance,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...


### 2022 data

In [210]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [212]:
# Peek into data downloaded from html

display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,7,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2]
1,J A N U A R Y,7,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,[3]
2,J A N U A R Y,7,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4]
3,J A N U A R Y,7,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,[5]
4,J A N U A R Y,14,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,A P R I L,1,Morbius,Columbia Pictures / Marvel Entertainment / A2 ...,"Daniel Espinosa (director); Matt Sazama, Burk ...",[65]
1,A P R I L,1,The Bubble,Netflix / Apatow Productions,Judd Apatow (director/screenplay); Pam Brady (...,[66]
2,A P R I L,1,The Contractor,Paramount Pictures / STXfilms / Thunder Road F...,Tarik Saleh (director); J. P. Davis (screenpla...,[67]
3,A P R I L,1,Better Nate Than Ever,Disney+ / Walt Disney Pictures,Tim Federle (director/screenplay); Aria Brooks...,[68]
4,A P R I L,1,Apollo 10 1⁄2: A Space Age Childhood,Netflix / Netflix Animation,Richard Linklater (director/screenplay); Glen ...,[69]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J U L Y,1,Minions: The Rise of Gru,Universal Pictures / Illumination,Kyle Balda (director); Matthew Fogel (screenpl...,[125]
1,J U L Y,1,The Forgiven,Roadside Attractions / Vertical Entertainment ...,John Michael McDonagh (director/screenplay); R...,[126]
2,J U L Y,1,Mr. Malcolm's List,Bleecker Street / Ingenious Media / Refinery29,Emma Holly Jones (director); Suzanne Allain (s...,[127]
3,J U L Y,1,The Princess,Hulu / 20th Century Studios / Original Film,"Le-Van Kiet (director); Ben Lustig, Jake Thorn...",[128]
4,J U L Y,1,Hot Seat,Lionsgate,James Cullen Bressack (director); Collin Watts...,[129]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,O C T O B E R,2.0,Mr. Harrigan's Phone,Netflix / Blumhouse Productions,John Lee Hancock (director/screenplay); Donald...,[180]
1,O C T O B E R,4.0,Terrifier 2,Bloody Disgusting / Fuzz on the Lens Productio...,Damien Leone (director/screenplay); Lauren LaV...,[200]
2,O C T O B E R,7.0,"Lyle, Lyle, Crocodile",Columbia Pictures / TSG Entertainment,"Will Speck, Josh Gordon (directors); William D...",[201]
3,O C T O B E R,7.0,Amsterdam,20th Century Studios / Regency Enterprises,David O. Russell (director/screenplay); Christ...,[202]
4,O C T O B E R,7.0,Tár,Focus Features,Todd Field (director/screenplay); Cate Blanche...,[203]


In [234]:
new_df22 = ProcessData(df1, df2, df3, df4, config_file = "config.yml")
new_df22.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Adventure Thriller,the 355,Jessica Chastain Penélope Cruz Fan Bingbing Si...
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Danny Trejo Antonio Cupo Patrici...
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,the commando,Mickey Rourke Michael Jai White unknown Asif A...
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,american siege,Timothy V. Murphy Bruce Willis Rob Gough Edwar...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...


### 2023 data

In [237]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [238]:
# Peek into data downloaded from html

display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,6,M3GAN,Universal Pictures / Blumhouse Productions / A...,Gerard Johnstone (director); Akela Cooper (scr...,[3]
1,J A N U A R Y,6,The Old Way,Saban Films / Saturn Films,Brett Donowho (director); Carl W. Lucas (scree...,[4]
2,J A N U A R Y,11,The Devil Conspiracy,Samuel Goldwyn Films,Nathan Frankowski (director); Ed Alan (screenp...,[5]
3,J A N U A R Y,13,Plane,Lionsgate / MadRiver Pictures / Di Bonaventura...,Jean-François Richet (director); Charles Cummi...,[6]
4,J A N U A R Y,13,House Party,Warner Bros. Pictures / New Line Cinema,"Calmatic (director); Jamal Olori, Stephen Glov...",[7]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,A P R I L,4,Miracle at Manchester,Bridgestone Media / JC Films,Eddie McClintock (director); Jason Campbell (s...,[72]
1,A P R I L,5,The Super Mario Bros. Movie,Universal Pictures / Illumination / Nintendo,"Aaron Horvath, Michael Jelenic (directors); Ma...",[73]
2,A P R I L,5,Air,Amazon Studios / Skydance Media / Mandalay Pic...,Ben Affleck (director); Alex Convery (screenpl...,[74]
3,A P R I L,7,Praise This,Peacock / Universal Pictures / Will Packer Pro...,Tina Gordon Chism (director/screenplay); Camil...,[75]
4,A P R I L,7,Showing Up,A24,Kelly Reichardt (director/screenplay); Jonatha...,[76]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J U L Y,4,Sound of Freedom,Angel Studios / Santa Fe Films,Alejandro Monteverde (director/screenplay); Ro...,[150]
1,J U L Y,7,Insidious: The Red Door,Screen Gems / Stage 6 Films / Blumhouse Produc...,Patrick Wilson (director); Scott Teems (screen...,[151]
2,J U L Y,7,Joy Ride,Lionsgate / Point Grey Pictures,Adele Lim (director); Cherry Chevapravatdumron...,[152]
3,J U L Y,7,The Out-Laws,Netflix / Happy Madison Productions,"Tyler Spindel (director); Evan Turner, Ben Zaz...",[153]
4,J U L Y,7,Earth Mama,A24 / Film4 Productions / Academy Films,Savanna Leaf (director/screenplay); Tia Nomore...,[154]


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,O C T O B E R,1,Oracle,Max / Universal Pictures / Will Packer Product...,"Daniel Di Grado (director); Michael Ross, Core...",[228]
1,O C T O B E R,2,Appendage,Hulu / 20th Digital Studio,Anna Zlokovic (director/screenplay); Hadley Ro...,[229]
2,O C T O B E R,6,The Exorcist: Believer,Universal Pictures / Blumhouse Productions / M...,David Gordon Green (director/screenplay); Pete...,[230]
3,O C T O B E R,6,Pet Sematary: Bloodlines,Paramount+ / Paramount Players / Di Bonaventur...,Lindsey Anderson Beer (director/screenplay); J...,[231]
4,O C T O B E R,6,Totally Killer,Amazon Studios / Blumhouse Productions,"Nahnatchka Khan (director); David Matalon, Sas...",[232]


In [247]:
new_df23 = ProcessData(df1, df2, df3, df4, config_file = "config.yml")
new_df23.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,m3gan,Allison Williams Violet McGraw Amie Donald Ger...
1,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,unknown,Western Drama,the old way,Nicolas Cage Ryan Kiera Armstrong unknown Bret...
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,the devil conspiracy,Alice Orr-Ewing Joe Doyle Eveline Hall Nathan ...
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,plane,Gerard Butler Mike Colter Yoson An Jean-Franço...
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,house party,Tosin Cole Jacob Latimore Karen Obilom Calmati...


In [303]:
df_latest = pd.concat([new_df23, new_df22, new_df21, new_df20, new_df19, new_df18], axis = 0, ignore_index=True)
df_latest.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,m3gan,Allison Williams Violet McGraw Amie Donald Ger...
1,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,unknown,Western Drama,the old way,Nicolas Cage Ryan Kiera Armstrong unknown Bret...
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,the devil conspiracy,Alice Orr-Ewing Joe Doyle Eveline Hall Nathan ...
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,plane,Gerard Butler Mike Colter Yoson An Jean-Franço...
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,house party,Tosin Cole Jacob Latimore Karen Obilom Calmati...


In [305]:
df_old = pd.read_csv('./new_data.csv')
df_old.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...


In [307]:
df_final = pd.concat([df_latest, df_old], axis=0, ignore_index = True)

In [312]:
df_final.shape

(7158, 7)

In [314]:
# Delete duplicate records
df_final.drop_duplicates(subset = "movie_title", keep = 'last', inplace=True)


In [317]:
# Check for missing values
df_final.isnull().sum()

director_name    29
actor_1_name      0
actor_2_name      0
actor_3_name      0
genres            1
movie_title       0
comb             30
dtype: int64

In [320]:
# Drop records with missing values
df_final = df_final.dropna(how='any')

In [324]:
# Check for missing values
df_final.isnull().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [326]:
df_final.to_csv('./main_data.csv',index=False)

### OLD CODE

In [250]:
# # Append all 4 dataframes together

# df = pd.concat([df1, df2, df3, df4], axis = 0, ignore_index=True)

In [252]:
# df.head()

In [254]:
# # Get TMDB API key
# # get your API Key here: https://www.themoviedb.org/settings/api by creating an account and requesting API access

# with open("config.yml", "r") as f:
#     config = yaml.safe_load(f)

# api_key = config["api_key"]

In [256]:

# tmdb = TMDb()
# tmdb.api_key = api_key 

In [258]:
# tmdb_movie = Movie()
# def get_genre(x): #pass in the title of the movies
#     genres = []
#     result = tmdb_movie.search(x) #the title will be searched in the tmdb_movie
#     movie_id = result[0].id #we will match the "id" with the "title"
#     response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key)) #we will get the result from the IMDb data
#     data_json = response.json() #we will then convert it to a json file
#     if data_json['genres']: #in the json file we will only need to extract the "genre"
#         genre_str = " " 
#         for i in range(0,len(data_json['genres'])):
#             genres.append(data_json['genres'][i]['name']) #we will then add the "genre" to the empty genre list we created above
#         return genre_str.join(genres)
#     else:
#         np.NaN # we will return the results but if we don't find anything we will consider it as a missing value


In [260]:
# df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))
# df.head()

In [262]:
# # Subset to only required columns

# df_2018 = df[['Title', 'Cast and crew', 'genres']].copy()
# df_2018.head()

In [264]:
# # Function to get directors name

# def get_director(x):
#     if " (director)" in x: # we are getting "director"
#         return x.split(" (director)")[0]
#     elif " (directors)" in x: # we are getting "directors" with "s"
#         return x.split(" (directors)")[0]
#     elif " (director[" in x:
#         return x.split(" (director[")[0]
#     elif " (director/" in x:
#         return x.split(" (director/")[0] # we are getting "director/screenplay"

In [266]:
# #let's apply the above function on the "Cast and crew" column
# df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))
# df_2018.head()

In [268]:
# df_2018['director_name'] = df_2018['director_name'].str.replace(" (director/screenplay);",",")

In [270]:
# df_2018.head()

In [272]:
# def get_actor1(x):
#     return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [274]:
# df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))
# df_2018.head()

In [276]:
# df_2018['actor_1_name'] =df_2018['actor_1_name'].str.replace("Lauren Wolkstein (director); ", "")

In [278]:
# def get_actor2(x):
#     if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
#         return np.NaN
#     else:
#         return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [280]:
# df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))
# df_2018.head()

In [282]:

# def get_actor3(x):
#     if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
#         return np.NaN
#     else:
#         return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [284]:
# df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))
# df_2018.head()

In [228]:
# df_2018 = df_2018.rename(columns={'Title':'movie_title'})

In [288]:
# new_df18 = df_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
# new_df18.head()

In [286]:
# new_df18['actor_1_name'] = new_df18['actor_1_name'].replace(np.nan, 'unknown')
# new_df18['actor_2_name'] = new_df18['actor_2_name'].replace(np.nan, 'unknown')
# new_df18['actor_3_name'] = new_df18['actor_3_name'].replace(np.nan, 'unknown')
# new_df18['movie_title'] = new_df18['movie_title'].str.lower()
# new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']
# new_df18.head()